### Installation:

pip install matplotlib

pip install rasterio

### Automation in accessing
    - local folders
    - clipping to extent,
    - resample
    - save as renamed .tiff
    - export to dependancy folder

In [5]:
import rasterio.warp
from rasterio.crs import CRS
import shapely.geometry as sg
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterio import mask
import numpy as np
import os

In [6]:

# Specify the input folder path
root_folder = 'Data/'
#input_folder = "Data/temp_mean/"  # Update with your input folder path
target_resolution = 250

# Specify the output folder path
output_folder = "Data_automated"  # Update with your desired folder path

# Retrieve a list of input file paths from the input folder
def get_image_files(root_folder, file_extension='.tif'):
    image_files = []

    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(file_extension):
                relative_path = os.path.relpath(root, root_folder)  # Get the relative path of the subfolder
                relative_path = relative_path.replace('/\\', '/')  # Convert backslashes to forward slashes
                output_subfolder = os.path.join(output_folder, relative_path)  # Create the output subfolder path
                os.makedirs(output_subfolder, exist_ok=True)  # Create the output subfolder if it doesn't exist
                image_files.append((os.path.join(root, file), output_subfolder))  # Append the file and output subfolder tuple

    return image_files

# Function to generate the output file name based on the input file name
def generate_output_filename(input_filename):
    base_name = os.path.basename(input_filename)
    filename, extension = os.path.splitext(base_name)
    output_filename = "resampled_image_" + filename + ".tif"
    return output_filename

image_files = get_image_files(root_folder)

# Process each image file and save the output in the respective subfolder
for file, output_subfolder in image_files:
    image_path = file
    # Perform the processing and save the output TIFF file in the output folder
    
    # Define the bounding box coordinates
    # Define the region of interest (Western Cape, South Africa)
    roi = [18.723629, -34.050015, 19.390929, -33.370358]
    left, bottom, right, top = roi
    
    # Specify the target coordinate reference system (CRS) for reprojection
    target_crs = 'EPSG:4326'
    target_resolution = target_resolution  # Update with your desired target resolution in meters

    ##############
    # Read Array and MetaData
    ##############
    # Function to read local raster data and return as a variable
    def read_local(image_path):
        with rasterio.open(image_path) as src:
            array = src.read()
            image_meta = src.meta
            image_meta['crs'] = str(src.crs)
            image_meta['width'] = src.width
            image_meta['height'] = src.height
            image_meta['bounds'] = src.bounds
        return array, image_meta
    # Read the local raster image
    image_array, image_meta = read_local(image_path)
    
    # Clip the image array to the ROI
    with rasterio.open(image_path) as src:
        roi_polygon = sg.Polygon([(roi[0], roi[1]), (roi[2], roi[1]), (roi[2], roi[3]), (roi[0], roi[3])])
        clipped, transform = rasterio.mask.mask(src, [roi_polygon], crop=True)
        clipped_meta = src.meta.copy()
        # Print the bounds of the original image and the clipped image
        #print("Original Image Bounds:", src.bounds)

        # Calculate the bounds of the clipped image
        clipped_height, clipped_width = clipped.shape[1:]
        clipped_bounds = rasterio.transform.array_bounds(clipped_height, clipped_width, transform)

        # Print the bounds of the clipped image
        #print("Clipped Image Bounds:", clipped_bounds)
    
    # Calculate the roi_transform
    roi_transform = transform

    ##############
    # Reproject & Resample
    ##############
    # Define the target CRS
    target_crs = CRS.from_string(target_crs)
    height = clipped_meta['height']
    width = clipped_meta['width']
    
    # Calculate the transform and dimensions of the resampled image
    resample_transform, resample_width, resample_height = calculate_default_transform(
        clipped_meta['crs'], target_crs,
        width, height,
        left, bottom, right, top,
        dst_width=270, dst_height=275  # Update the desired number of columns and rows here
    )

    # Update the metadata with the resampled transform and dimensions
    clipped_meta.update({
        'transform': resample_transform,
        'width': resample_width,
        'height': resample_height
    })

    # Create the reprojected array
    reprojected_array = np.zeros((clipped.shape[0], target_resolution, target_resolution), dtype=clipped.dtype)


    # Reproject the clipped image array
    reproject(
        source=clipped,
        destination=reprojected_array,
        src_transform=transform,
        src_crs=clipped_meta['crs'],
        dst_transform=resample_transform,
        dst_crs=target_crs,
        resampling=Resampling.nearest
    )

    # Update the metadata of the reprojected array
    reprojected_meta = clipped_meta.copy()
    reprojected_meta['transform'] = resample_transform
    reprojected_meta['width'] = resample_width
    reprojected_meta['height'] = resample_height

    # Create a new file for the resampled image
    output_filename = generate_output_filename(image_path)
    output_path = os.path.join(output_subfolder, output_filename)

    # Write the resampled image to the output file
    with rasterio.open(output_path, 'w', **reprojected_meta) as dst:
        dst.write(reprojected_array)
        print(output_filename)


resampled_image_250m_Extent_WGS84.tif
resampled_image_2023-01-01_40m_k30.tif
resampled_image_2023-01-02_40m_k30.tif
resampled_image_2023-01-01_40m_k30.tif
